# Imports

In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime

# Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Creating Data Frames

Since our data files are all in the `.dat` format it will be better to save them as `.csv`. We can also take time to process some of the columns to make analysis easier later on. In simplistic terms, I will be replacing some of the numeric values to their string equivalents based off the `README` that came with the 1-m dataset.

In [ ]:
# Creating the path of data
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/one-m-capstone/data/'

# Movies Data

The movies data is fine as is, we need only add column names and save as a `.csv`.

In [ ]:
movies = pd.read_csv(DATA_PATH + 'movies.dat', sep='::', engine='python', names=['movie_id', 'title', 'genres'])

In [ ]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  3883 non-null   int64 
 1   title     3883 non-null   object
 2   genres    3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


In [ ]:
# Break up the big genre string into a string array
movies['genres'] = movies['genres'].str.split('|')
# Convert genres to string value
movies['genres'] = movies['genres'].str.join(' ').astype(str)

In [ ]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation Children's Comedy
1,2,Jumanji (1995),Adventure Children's Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama
4,5,Father of the Bride Part II (1995),Comedy


# Ratings Data


In [ ]:
ratings = pd.read_csv(DATA_PATH + 'ratings.dat', sep='::', 
                     engine='python', 
                     names=['user_id', 'movie_id', 'rating', 'timestamp'])

In [ ]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   user_id    1000209 non-null  int64
 1   movie_id   1000209 non-null  int64
 2   rating     1000209 non-null  int64
 3   timestamp  1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


In [ ]:
# Each user has 20 ratings
ratings.groupby('user_id')['rating'].sum().sort_values(ascending=False)

user_id
4169    8219
4277    7207
1680    6578
1941    4872
2909    4809
        ... 
5828      54
4349      53
4365      51
4056      51
4636      50
Name: rating, Length: 6040, dtype: int64

# Users Data


In [ ]:
users = pd.read_csv(DATA_PATH + 'users.dat', 
                    sep='::', 
                    engine='python', 
                    names=['user_id', 'gender', 'age', 'occupation', 'zip'])

In [ ]:
users.head()

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [ ]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     6040 non-null   int64 
 1   gender      6040 non-null   object
 2   age         6040 non-null   int64 
 3   occupation  6040 non-null   int64 
 4   zip         6040 non-null   object
dtypes: int64(3), object(2)
memory usage: 236.1+ KB


## Elaborating on the numeric values

In [ ]:
age_map = {
    1  : 'Under 18',
    18 : '18-24',
    25 : '25-34',
    35 : '35-44',
    45 : '45-49',
    50 : '50-55',
    56 : '56+'
} 

occ_map = {
    0:  "other or not specified",
    1:  "academic/educator",
    2:  "artist",
    3:  "clerical/admin",
    4:  "college/grad student",
    5:  "customer service",
    6:  "doctor/health care",
    7:  "executive/managerial",
    8:  "farmer",
    9:  "homemaker",
    10:  "K-12 student",
    11:  "lawyer",
    12:  "programmer",
    13:  "retired",
    14:  "sales/marketing",
    15:  "scientist",
    16:  "self-employed",
    17:  "technician/engineer",
    18:  "tradesman/craftsman",
    19:  "unemployed",
    20:  "writer"
}

In [ ]:
users['age_elab'] = users['age'].map(age_map)
users['occ_elab'] = users['occupation'].map(occ_map)

In [1]:
users.head()

NameError: ignored

# Combining Data Frames

The combined dataframe is an expanded ratings folder joining the other two dataframes. This makes it easier to manipulate the data and put it into a model for matrix factorization.

In [ ]:
df = pd.merge(movies, ratings, on='movie_id')
df = pd.merge(df, users, on='user_id')

In [ ]:
df.head()

,movie_id,title,genres,user_id,rating,timestamp,gender,age,occupation,zip,age_elab,occ_elab
0,1,Toy Story (1995),Animation Children's Comedy,1,5,978824268,F,1,10,48067,Under 18,K-12 student
1,48,Pocahontas (1995),Animation Children's Musical Romance,1,5,978824351,F,1,10,48067,Under 18,K-12 student
2,150,Apollo 13 (1995),Drama,1,5,978301777,F,1,10,48067,Under 18,K-12 student
3,260,Star Wars: Episode IV - A New Hope (1977),Action Adventure Fantasy Sci-Fi,1,4,978300760,F,1,10,48067,Under 18,K-12 student
4,527,Schindler's List (1993),Drama War,1,5,978824195,F,1,10,48067,Under 18,K-12 student


# Cleaning Movies that Haven't Been Rated

These movies will be particularly troublesome for our recommendatoin engine, as we need some basis to recommend them. Since we are building our own engine, deleting them is not advisable. Instead we will impute the values of the movie by giving the a singular review each that is the average of the rating in the genre. We will also create a test user and add it to our users csv from which the ratings will originate.

## Unnrated Movies

In [ ]:
all_movie_ids = movies['movie_id'].values
rated_movie_ids = ratings['movie_id'].values
unrated_ids = [x for x in all_movie_ids if x not in rated_movie_ids]

print(f'There are  {len(unrated_ids)}  movies out of that do not have ratings')

There are  177  movies out of that do not have ratings


## Creating Fake User

In [ ]:
# creating a fake user that we can use to make our test predictions in ratings from
fake_user = pd.Series([6041, 'M', 30, 0, '07974', '25-34', 'other or not specified'], index=users.columns)
users = users.append(fake_user, ignore_index=True)
# only run once
#users.to_csv(DATA_PATH + 'processed_users.csv')

## Creating Fake (Imputed) Ratings

In [ ]:
# First create a mapping of genre to average ratings
ugenre_map = dict(df.groupby('genres')['rating'].mean())

# create a fake ratings dataframe
fake_ratings = []

for movie_id in unrated_ids:
  # check the movies genre and get the genre's average rating
  movie_series = movies.query(f'movie_id == {movie_id}')
  genre = str(movie_series['genres'].values).strip('[]').replace("'",'')
  
  # if genre not listed just use 3... which is the global average
  try:
    g_avg = ugenre_map[genre]
  except:
    g_avg = 3

  # our fake user from above will be the user submitting the rating
  user_id = 6041

  # we need a timestamp for the rating data
  time_stamp = int(datetime.timestamp(datetime.now()))

  # create the rating to attach
  fake_rate = pd.Series([user_id, movie_id, g_avg, time_stamp], index=['user_id','movie_id','rating','timestamp'])
  fake_ratings.append(fake_rate.astype(int).values)

In [ ]:
fakedf = pd.DataFrame(data=fake_ratings, columns=['user_id','movie_id','rating','timestamp'])

In [ ]:
ratings = ratings.append(fakedf,ignore_index=True)

## Re-merging dataframes

In [ ]:
df = pd.merge(movies, ratings, on='movie_id')
df = pd.merge(df, users, on='user_id')

## Checking Again to ensure the cleaning worked

In [ ]:
all_movie_ids = movies['movie_id'].values
rated_movie_ids = df['movie_id'].values
unrated_ids = [x for x in all_movie_ids if x not in rated_movie_ids]

print(f'There are  {len(unrated_ids)}  movies out of that do not have ratings')

There are  0  movies out of that do not have ratings


# Saving Data Frames


In [ ]:
movies.to_csv(DATA_PATH + 'processed_movies.csv', sep=',', header=True, columns=['movie_id', 'title', 'genres'])
ratings.to_csv(DATA_PATH + 'processed_ratings.csv', sep=',', header=True, columns=['user_id','movie_id','rating','timestamp'])
users.to_csv(DATA_PATH + 'processed_users.csv', sep=',', header=True, columns=['user_id', 'gender', 'age', 'occupation', 'zip', 'age_elab', 'occ_elab'])
df.to_csv(DATA_PATH + 'combined.csv', header=True)